In [1]:
import numpy as np 
import pdb 
import itertools

In [25]:
A = np.random.rand(1000,100)*1.0
B = np.random.rand(1000,200)*1000.0
U, S, V_ = np.linalg.svd(B, full_matrices=False)
# T = U.dot(np.diag(S))
# X_ideal, _, _, _ = np.linalg.lstsq(A,T)
X_ideal, _, _, _ = np.linalg.lstsq(A,B)

In [20]:
def CSPsolve(X_ideal, A, B, Z=None, generations=1):
    ''' 
    X_ideal (array-like): Unconstrained solution to AX = B
    A (array-like): Data Array 
    B (array-like): Solution Array
    Y (vector or list): Vector of all available numbers covered by basis
    '''
    m, n = X_ideal.shape
    basis0 = []
    for i in range(n):
        basis0.append(np.array([1,2,4,-8]))
    Z0 = CSPgenZ(basis0)
    Y_hi, Y_lo = CSPround(X_ideal, Z0)
    Y0 = np.minimum(Y_hi, Y_lo)
    Y0, loss0 = CSPeval(X_ideal, A, B, Y0=Y0, Z=Z0)
    _, loss_init = CSPeval(X_ideal, A, B, Y0=Y0, Z=Z0)
    X0 = Y0 + X_ideal
    for i in range(generations):
        Basis_set = {}
        Z = {}
        Y = {}
        Loss = []
        for j in range(10):
            basis_temp = []
            for b0 in basis0:
                b1 = b0 + np.random.randint(-5,5,4)
                basis_temp.append(b1)
            Z_temp = CSPgenZ(basis_temp)
            Basis_set[j] = basis_temp
            Z[j] = Z_temp 
            y, loss = CSPeval(X_ideal, A, B, Y0, Z=Z_temp)
            Y[j] = y
#             pdb.set_trace()
            Loss.append(loss)
            print('Epoch %r, generation %r complete' %(i+1, j+1))
#         Loss
        best_loss = np.min(Loss)
#         pdb.set_trace()
        idx = np.argmin(Loss)#TODO Fixme 
#         pdb.set_trace()
        if best_loss < loss0:
            loss0 = best_loss
            basis0 = Basis_set[idx]
            Y0 = Y[idx]
            
    basis = basis0
    Y = Y0    
    loss = loss0
    
    return [Y, basis, loss_init, loss]

In [21]:
def CSPgenZ(basis_set):
    Z = []
    for basis in basis_set: 
        coverage = [0]
        for i in range(3):
            pset = itertools.combinations(basis, i+1)
            for comb in pset: 
                val = np.sum(comb)
                coverage.append(val)
        z = np.unique(coverage)
        Z.append(z)
    return Z

In [22]:
def CSPeval(X_ideal, A, B, Y0=None, Z=None):
    m, n = X_ideal.shape
    if Y0 is None: 
        X0 = np.round(X_ideal)
        Y0 = X0 - X_ideal
    dL = X_ideal.T.dot(A.T.dot(A)) - B.T.dot(A) + Y0.T.dot(A.T.dot(A))
    Y = np.zeros_like(X_ideal)
    if Z is None: 
        Y_hi = np.ceil(X_ideal) - X_ideal
        Y_lo = np.floor(X_ideal) - X_ideal
    else: 
        Y_hi, Y_lo = CSPround(X_ideal, Z)
#     pdb.set_trace()
    dL_hi = np.abs(np.multiply(dL.T, Y_hi))
    dL_lo = np.abs(np.multiply(dL.T, Y_lo))
    mask_lo = dL_hi > dL_lo
    mask_hi = dL_hi < dL_lo
    Y = np.multiply(mask_lo, Y_lo) + np.multiply(mask_hi, Y_hi)
    X = X_ideal + Y 
    loss = 0.5*np.linalg.norm(A.dot(X) - B)**2
    return Y, loss 

In [23]:
def CSPround(X_ideal, Z):
    m, n = X_ideal.shape
    X_hi = np.zeros_like(X_ideal)
    X_lo = np.zeros_like(X_ideal)
    found_hi = np.ones_like(X_ideal)
    found_lo = np.ones_like(X_ideal)
    for j, z in enumerate(Z): 
        X_ = np.zeros([m,len(z)])
        for k, z_ in enumerate(z):
            X_[:,k] = X_ideal[:,j] - z_
        mask_neg = X_ < 0
        mask_pos = X_ > 0 
        X_neg = np.multiply(mask_neg, X_)
        X_neg[X_neg==0] = 10000.0
        X_pos = np.multiply(mask_pos, X_)
        X_pos[X_pos==0] = 10000.0
        idx_hi = np.argmin(np.abs(X_neg), axis=1)
        idx_lo = np.argmin(X_pos, axis=1) 
        X_hi[:, j] = z[idx_hi]
        X_lo[:, j] = z[idx_lo]
    Y_hi = X_hi - X_ideal 
    Y_lo = X_lo - X_ideal 
    return Y_hi, Y_lo
        

# def CSPround(X_ideal, Z):
#     m, n = X_ideal.shape
#     X_hi = np.zeros_like(X_ideal)
#     found_hi = np.ones_like(X_ideal)
#     X_lo = np.zeros_like(X_ideal)
#     found_lo = np.ones_like(X_ideal)
#     for z in Z: 
#         mask_hi = X_ideal < z 
#         X_hi += np.multiply(found_hi, mask_hi)*z
#         found_hi = 1.0 - mask_hi
#     for z in Z.reverse():
#         mask_lo = X_ideal > z
#         X_lo += np.multiply(found_lo, mask_lo)*z
#         found_lo = 1.0 - mask_lo
#     final_lo = found_lo*np.max(Z)
#     final_hi = found_hi*np.min(Z)
#     X_hi += final_hi
#     X_lo += final_lo
#     Y_hi = X_hi - X_ideal
#     Y_lo = X_lo - X_ideal
#     return Y_hi, Y_lo

In [26]:
Y, basis, loss_init, loss_final = CSPsolve(X_ideal, A, B, Z=None, generations=1)
print('Loss initial: %r, loss final: %r' % (loss_init, loss_final))

Epoch 1, generation 1 complete
Epoch 1, generation 2 complete
Epoch 1, generation 3 complete
Epoch 1, generation 4 complete
Epoch 1, generation 5 complete
Epoch 1, generation 6 complete
Epoch 1, generation 7 complete
Epoch 1, generation 8 complete
Epoch 1, generation 9 complete
Epoch 1, generation 10 complete
Loss initial: 62821360763.925026, loss final: 62821360763.925026


In [55]:
np.argmin.__doc__

'\n    Returns the indices of the minimum values along an axis.\n\n    Parameters\n    ----------\n    a : array_like\n        Input array.\n    axis : int, optional\n        By default, the index is into the flattened array, otherwise\n        along the specified axis.\n    out : array, optional\n        If provided, the result will be inserted into this array. It should\n        be of the appropriate shape and dtype.\n\n    Returns\n    -------\n    index_array : ndarray of ints\n        Array of indices into the array. It has the same shape as `a.shape`\n        with the dimension along `axis` removed.\n\n    See Also\n    --------\n    ndarray.argmin, argmax\n    amin : The minimum value along a given axis.\n    unravel_index : Convert a flat index into an index tuple.\n\n    Notes\n    -----\n    In case of multiple occurrences of the minimum values, the indices\n    corresponding to the first occurrence are returned.\n\n    Examples\n    --------\n    >>> a = np.arange(6).reshape

In [22]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

In [40]:
# X = X_ideal + V
X_round = np.round(X_ideal)
loss = 0.5*np.linalg.norm((A.dot(X)) - B)**2
loss_round = 0.5*np.linalg.norm((A.dot(X_round)) - B)**2
print('My Loss:', loss)
print('Naive Loss:', loss_round)

My Loss: 76028283.0431
Naive Loss: 76028283.0431


In [51]:
basis0 = np.tile(np.array([1,2,4,-8]).reshape([4,1]), 3)

In [52]:
basis0

array([[ 1,  1,  1],
       [ 2,  2,  2],
       [ 4,  4,  4],
       [-8, -8, -8]])

In [32]:

ddL = A.T.dot(A)
m, n = X_ideal.shape
V = np.zeros([m,n])
# X = X_ideal[:]
X_round = np.round(X_ideal)
V_round = X_round - X_ideal 
V = V_round

for k in range(1):
    dL = (A.T.dot(A)).dot(X_ideal + V_round) - A.T.dot(T)
    

# for k in range(1):
#     dL = (A.T.dot(A)).dot(X_ideal + V) - A.T.dot(T)
#     for j in range (n):
#         for i in range(m):
#             X_low = X[:]
#             X_hi = X[:]
#             X_low[i,j] = np.floor(X_ideal[i, j])
#             X_hi[i,j] = np.ceil(X_ideal[i,j])
#             L_low = 0.5*np.linalg.norm(A.dot(X_low) - T)**2
#             L_hi = 0.5*np.linalg.norm(A.dot(X_hi) - T)**2
#             if L_low < L_hi: 
#                 X[i, j] = X_low[i, j]
#             elif L_low > L_hi:
#                 X[i, j] = X_hi[i, j]
#             i = m - i_ - 1 
#             j = n - j_ -1 
#             V_low = V[:]
#             V_low[i, j] = np.floor(X_ideal[i, j]) - X_ideal[i, j]
#             V_hi = V[:]
#             V_hi[i, j] = np.ceil(X_ideal[i, j]) - X_ideal[i, j]
#             dL_low = np.trace(dL.T.dot(V_low)) + 0.5*np.trace((ddL.dot(V_low)).T.dot(V_low))
#             dL_hi = np.trace(dL.T.dot(V_hi)) + 0.5*np.trace((ddL.dot(V_hi)).T.dot(V_hi))
#             if dL_low < dL_hi: 
#                 V[i, j] = V_low[i, j]
#             elif dL_low > dL_hi:
#                 V[i, j] = V_hi[i, j]
    #         print('%r of % done' % (i + j*m + 1, m*n))

In [22]:
# X = X_ideal + V
X_round = np.round(X_ideal)
loss = 0.5*np.linalg.norm((A.dot(X)) - B)**2
loss_round = 0.5*np.linalg.norm((A.dot(X_round)) - B)**2
print('My Loss:', loss)
print('Naive Loss:', loss_round)

My Loss: 467788466.767
Naive Loss: 75954543.3103


In [54]:
A = np.array([[1,2,3],[4,5,6]])
B = np.array([0,1,2])
C = A > B


[[ True  True  True]
 [ True  True  True]]


In [59]:
np.argwhere(C).shape

(6, 2)

In [57]:
np.argwhere.__doc__

'\n    Find the indices of array elements that are non-zero, grouped by element.\n\n    Parameters\n    ----------\n    a : array_like\n        Input data.\n\n    Returns\n    -------\n    index_array : ndarray\n        Indices of elements that are non-zero. Indices are grouped by element.\n\n    See Also\n    --------\n    where, nonzero\n\n    Notes\n    -----\n    ``np.argwhere(a)`` is the same as ``np.transpose(np.nonzero(a))``.\n\n    The output of ``argwhere`` is not suitable for indexing arrays.\n    For this purpose use ``where(a)`` instead.\n\n    Examples\n    --------\n    >>> x = np.arange(6).reshape(2,3)\n    >>> x\n    array([[0, 1, 2],\n           [3, 4, 5]])\n    >>> np.argwhere(x>1)\n    array([[0, 2],\n           [1, 0],\n           [1, 1],\n           [1, 2]])\n\n    '

In [21]:
X_ideal

array([[-61.55801298,  22.17325017, -18.40437087,   0.60488248,
          3.74337513,  16.72365393,   2.79156629,  -3.38240778,
         12.88392325,  -2.22819074,  25.09074991,  15.70338883,
         12.43600441,   4.68726073, -11.01972697,  -1.09735278,
         -2.90309469,   5.17401219,   3.75082026,  10.54627833],
       [-66.57209632,  -2.26044067,   3.03974567,  10.6370506 ,
        -19.41435552,  -3.48854013,  21.59453364,   2.61537717,
          4.35344265, -11.07113351, -21.18376243,   6.58790328,
         -3.28970358,   9.5952105 ,   2.74617112, -10.09464122,
          0.12338762,  14.80253537,   2.12744681,  -1.91798091],
       [-58.06867326, -32.85752366,  24.68844444,  11.40431801,
        -19.08347967, -12.11439911,  18.30684964, -17.42444362,
         17.16073628,  18.04680908,   1.40477224,  14.41693095,
         -7.58077803,   9.59161566,  -1.92971244, -13.22399616,
         -9.31741281,  -9.03205754,  -2.93736507,  -2.08433488],
       [-50.30722512, -24.11834881, -